In [ ]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s

In [ ]:
import whisper
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
stt_model = whisper.load_model("base")  # You can use "tiny", "small", etc.
tokenizer = AutoTokenizer.from_pretrained("suriya7/t5-base-text-to-sql")
text2sql_model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/t5-base-text-to-sql")

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 39.3MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.mp3 to test.mp3


In [ ]:
audio_file = list(uploaded.keys())[0]
result = stt_model.transcribe(audio_file)
transcribed_text = result['text']
print("🎤 Transcribed Text:", transcribed_text)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🎤 Transcribed Text:  Find all available science fiction books by JK Rowling, published after 2010.


In [ ]:
schema = "ID | title | author | content | publisher | published_year | price | num_pages | status | location | genre"
prompt = f"translate English to SQL: {transcribed_text} | table columns = {schema}"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
outputs = text2sql_model.generate(**inputs, max_length=128)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("🧠 SQL Query:", sql_query)

🧠 SQL Query: SELECT available_science_fiction_books FROM table_name_69 WHERE published > 2010 | table_collars = ID | title | author | content | publisher | published_year | price | num_pages | status | location | genre
